In [35]:
# Goal 1

import csv
from collections import namedtuple


class ContextIterator:
    def __init__(self, fname):
        self._fname = fname
        self._file = None
        self._reader = None
        self._namedtuple = None

    def __iter__(self):
        return self

    def __next__(self):
        if not self._file.closed:
            return self._namedtuple(*next(self._reader))
        raise StopIteration
  
    def sniff_dialect(self):
        sample = self._file.read(1000)
        self._file.seek(0)
        return csv.Sniffer().sniff(sample)

    def __enter__(self):
        print('---')
        print(f'{self._fname} opeing...')
        print('---')

        self._file = open(self._fname)

        dialect = self.sniff_dialect()
        self._reader = csv.reader(self._file, dialect)

        header_row = map(lambda s: s.lower(), next(self._reader))
        self._namedtuple = namedtuple('NamedContextIter', header_row)
        
        return self

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        print('---')
        print(f'{self._fname} closing...')
        print('---')

        self._file.close()
        return False


# Goal 2

import csv
from collections import namedtuple

class Context:
    def __init__(self, gen):
        self._gen = gen

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False


def decorator(gen_func):
    def inner(*args, **kwargs):
        return Context(gen_func(*args, **kwargs))
    return inner


@decorator
def open_file(fname):
    print(f'{fname} opening')
    file = open(fname, 'r')
    try:
        yield file
    finally:
        print(f'{fname} closing')
        file.close()


def get_dialect(fname):
    with open(fname, 'r') as file:
        return csv.Sniffer().sniff(file.read(1000))


class ContextIterator:
    def __init__(self, fname):
        self._fname = fname

    def __iter__(self):
        return self

    def __next__(self):
        if not self._file.closed:
            return self._namedtuple(*next(self._reader))
        raise StopIteration
    
    def __enter__(self):
        self._ctx = open_file(self._fname)
        self._file = self._ctx.__enter__()
        self._reader = csv.reader(self._file, get_dialect(self._fname))
        header_row = map(lambda s: s.lower(), next(self._reader))
        self._namedtuple = namedtuple('Data', header_row)
        return self

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        self._ctx.__exit__(exc_type, exc_value, exc_trace_back)
        return False


class ContextIterators:
    def __init__(self):
        self._files = []
        self._readers = []
        self._exits = []
        self._namedtuples = []

    def __iter__(self):
        return self

    def __next__(self):
        for file in self._files:
            if file.closed:
                raise StopIteration
        return [_namedtuple(*next(reader))
                for _namedtuple, reader in
                zip(self._namedtuples, self._readers)]
        
    def __enter__(self):
        return self

    def enter_context(self, fname):
        self._ctx = open_file(fname)
        self._exits.append(self._ctx.__exit__)
        
        file = self._ctx.__enter__()
        self._files.append(file)
        reader = csv.reader(file, get_dialect(fname))
        self._readers.append(reader)

        header_row = map(lambda s: s.lower(), next(reader))
        self._namedtuples.append(namedtuple('Data', header_row))

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_trace_back)
        return False


fnames = 'personal_info.csv', 'cars.csv'

with ContextIterators() as stack:
    for fname in fnames:
        stack.enter_context(fname)
    for data in next(stack):
        print(data)



personal_info.csv opening
cars.csv opening
Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
cars.csv closing
personal_info.csv closing
